> **本节概要：** 从贝叶斯的角度考虑模型的选择问题


对于回归模型和他的MSE：
$$
y(x,w) = \sum^{M-1}_{j=0}w_j\phi_j(x)=w^T\phi(x) \\
E_D(w)=\frac{1}{2}\sum_{n=1}^N{t_n-w^T\phi(x_n)}^2
$$

- 我们需要确定的是参数M，这个参数决定了模型的复杂度。
- 如果我们加入正则项 $E(w)=\frac{1}{2}\sum_{n=1}^N{t_n-w^T\phi(x_n)}^2+\frac{\lambda}{2}w^Tw$。正则项的$\lambda$也控制着模型的复杂度。

---
## 从频率派的观点选择模型
---
- **训练集：** 训练不同的$M$和$\lambda$确定系数$w$
- **验证集：** 优化模型的复杂度（$M$和$\lambda$）

<img src="https://raw.githubusercontent.com/data2world/PRML_Note/master/IMG/CH01/1.8.png" style="width:500px;height:300;">

如上图， 

- $\lambda = -38$: 蓝色的训练集的error很小，但是验证集的error很大
- $\lambda = -30$: 模型的效果很好

当然，如何正确选择验证集，也是一个问题。当数据不足时，一个常用的方法就是交叉验证，用交叉验证来重复使用数据，切分，反复训练、测试、模型选择。交叉验证也是有缺点的，例如：切分数据集过多，则训练时间过久；不同的数据集会返回不同的复杂度参数，参数的组合类型是指数型增长的。

另外，AIC(赤池信息准则)也可以进行模型的选择。(其他章节有介绍)

---
## 从贝叶斯的观点选择模型
---
贝叶斯观点选择模型，就是使用Model Evidence来进行选择，Model Evidence也可以叫做Marginal Likelihood（边缘似然）。在贝叶斯派来看，为什么不能直接使用最大似然函数作为模型的选择？其实是因为最大似然函数的拟合能力太强，容易导致模型过拟合，所以使用边缘似然函数。它表达了数据展现出的不同模型的优先级。

边缘似然 $P(D|M)$，它实际上可以用上面的预测分布连乘来表示：
$$
P(D|M) =
P(x_1,x_2,...,x_n|M)=
P(x_1|M)P(x_2|x_1,M)P(x_3|x_1,x_2,M)..P(x_n|x_{1..n-1},M)
$$

这个过程可以理解为，我们先计算模型生成$x_1$的概率，然后乘以$x_1$为训练集时$x_2$的预测分布，依次类推。显然，如果一个模型过于复杂，那么预测分布值会较小（因为预测性能不好），那么在连乘后，得到的边缘似然也很小。

- ⼀个模型指的是观测数据D上的概率分布。
- 假设我们想⽐较$L$个模型$M_i$，其中$i=1,...,L$。
- 我们假设数据是由这些模型中的一个生成的，但不知道究竟是哪一个 。
- 这种不确定性通过先验概率分布$p(M_i)$表示，现在我们想估计对应的后验分布：

$$
p(M_{i}|D)\propto p(M_{i})p(D|M_{i})
$$


### 边缘似然的近似法
对边缘似然函数$P(D|M_i)$
- $贝叶斯因子=\frac{先验概率比}{后验概率比}$, $bayes factor = \frac{p(D|M_i)}{p(D|M_j)}$

对模型$M_i$, 假设给一个单独的参数$w$,那么：
$$
p(D|M_{i})=\int p(D|w, M_{i})p(w|M_{i})dw
$$

对边缘似然函数，参数$w$的后验分布为：
$$
p(w|D,M_i)=\frac{p(D|w,M_i)p(w|M_i)}{p(D|M_i)}
$$

下面考虑一种对积分的近似方法。假设后验分布在最⼤似然值$w_MAP$附近是⼀个尖峰，宽度为$\Delta{w}$后验，那么可以⽤被积函数的值乘以尖峰的宽度来近似这个积分。如果进⼀步假设先验分布是平的，宽度为$\Delta{w}$先验，即$p(w)=\frac{1}{\Delta{w}}$先验，那么有:

$$
p(D)=\int p(D|w)p(w)dw\simeq p(D|w_{MAP})\frac{\triangle w_{后验}}{\triangle w_{先验}} \\
ln\ p(D)\simeq ln\ p(D|w_{MAP})+ln\ \frac{\triangle w_{后验}}{\triangle w_{先验}}
$$

这里，第⼀项表⽰拟合由最可能参数给出的数据。对于平的先验分布来说，这对应于对数似然。第⼆项⽤于根据模型的复杂度来惩罚模型
![](https://raw.githubusercontent.com/data2world/PRML_Note/master/IMG/CH03/3.12.png)

对$M$个参数的模型：
$$
ln\ p(D)\simeq ln\ p(D|w_{MAP})+M ln\ \frac{\triangle w_{后验}}{\triangle w_{先验}}
$$

模型的惩罚度随着$M$的增大线性增加。随着我们增加模型的复杂度，第⼀项通常会增⼤，因为⼀个更加复杂的模型能够更好地拟合数据，⽽第⼆项会减⼩，因为它依赖于$M$。由最⼤模型证据确定的最优的模型复杂度需要在这两个相互竞争的项之间进⾏折中。

![](https://raw.githubusercontent.com/data2world/PRML_Note/master/IMG/CH03/3.13.png)

**贝叶斯因子总是倾向于正确的模型。**

- 如果$M_1$是好的模型，那么贝叶斯因子就是：
$$
\int p(D|M_1)\ln \frac{p(D|M_1)}{p(D|M_2}\mathrm{dD}
$$
